SIDE NOTE: would love to get any critiques, hints, tips, oberservations on any level not just on ML (but obviously ML as the priority).  Beginner Python learner (though I took the Full Stack Course) so any suggestions there are welcome.  Thanks.

Will need to import these libs plus setting some global variables

In [30]:
import sys
import os
import numpy as np
import renders as rs
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from IPython.display import display # Allows the use of display() for DataFrames
from sklearn.externals import joblib
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
from sklearn.cross_validation import StratifiedKFold
from sklearn.svm import SVC
import matplotlib.cm as cm
# Show matplotlib plots inline (nicely formatted in the notebook)
%matplotlib inline

# Might need to change the path of the included libraries.
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats/match_stats.py')
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats/form_model.py')
sys.path.append('/anaconda/envs/stats/lib/python3.5/site-packages')
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats/model_libs.py')
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats/form_data.py')
sys.path.append('/Users/senzari/Machine_Learning/stats')
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats')
# print(sys.path)
os.chdir('/Users/senzari/Machine_Learning/stats/src')
#print(os.getcwd())

from stats import form_data, match_stats, model_libs, form_model, predict_matches

# Variables
round_number = 27 # for MLS only
target_col = 'points'
ignore_cols = ['match_id', 'team_id', 'team_name', 'opp_id', 'opp_name', 'scheduled', 'games_played', 'round', 'current_formation']
all_models = ['log', 'svc', 'knn', 'gnb', 'randomForest', 'gmm']
leagues = model_libs.get_leagues_country_codes()
#leagues = { "epl": 'ENG' }
teams = form_data.get_teams()
league_rounds = model_libs.get_leagues_rounds()
target = "points"

""" this variable 'testing' should be false if using CSV's and not pulling from the database. """
testing = False

In [2]:
data_csv = 'raw_data.csv'

if testing:
    raw_data = form_data.run_data()
    raw_data.to_csv(data_csv)
    print("Raw Data Saved to CSV")
else:
    #Reading in a CSV adds the first index column
    raw_data = pd.read_csv(data_csv)
    raw_data = raw_data.drop(raw_data.columns[[0]], axis=1)

pd.set_option("display.max_columns", 85)
print('Data Loaded...')
print("Dataset size :: {}".format(raw_data.shape))
display(raw_data.head())

Data Loaded...
Dataset size :: (634, 47)


,match_id,team_id,team_name,opp_id,opp_name,scheduled,round,games_played,is_home,current_formation,current_record,opp_record,goals_for,opp_goals_for,goals_against,opp_goals_against,rpi,goals,points,current_team_possession,current_team_yellow_cards,current_team_goal_attempts,current_team_dangerous_attacks,current_team_sec_half_goals,current_team_saves,current_team_corner_kicks,current_team_ball_safe,current_team_first_half_goals,current_team_shots_on_target,current_team_attacks,current_team_goal_attempts_allowed,current_team_goal_kicks,current_team_shots_total,opp_team_possession,opp_team_yellow_cards,opp_team_goal_attempts,opp_team_dangerous_attacks,opp_team_sec_half_goals,opp_team_saves,opp_team_corner_kicks,opp_team_ball_safe,opp_team_first_half_goals,opp_team_shots_on_target,opp_team_attacks,opp_team_goal_attempts_allowed,opp_team_goal_kicks,opp_team_shots_total
0,249,21,FC Dallas,33,DC United,2016-03-26 21:30:00,4,3,0,4-2-3-1,0.666667,0.357143,4,2,5,5,0.630952,3,3,151,6,35,182,3,5,20,256,1,20,310,16,17,35,138,8,22,150,1,4,16,283,1,11,293,20,23,22
1,255,21,FC Dallas,39,Columbus Crew,2016-04-03 01:00:00,5,3,1,4-2-3-1,0.714286,0.166667,5,2,5,4,0.465919,1,1,152,7,32,169,3,5,14,284,2,17,336,19,16,32,171,3,39,194,2,12,22,248,0,18,289,22,15,37
2,265,21,FC Dallas,32,San Jose Earthquakes,2016-04-10 01:00:00,6,3,1,4-4-2,0.833333,0.500000,6,4,1,5,0.547723,2,1,148,6,45,168,4,9,10,286,2,19,339,19,17,45,145,7,19,151,2,7,11,308,2,11,309,28,31,19
3,272,21,FC Dallas,30,Portland Timbers,2016-04-14 02:30:00,7,3,0,4-4-2,0.687500,0.357143,6,4,3,7,0.497669,3,3,154,8,41,145,3,8,15,270,3,17,310,21,16,41,143,8,24,164,4,9,27,273,0,11,300,23,19,24
4,282,21,FC Dallas,27,Sporting Kansas City,2016-04-17 23:00:00,7,3,1,4-4-2,0.687500,0.625000,6,4,3,2,0.497669,2,3,154,8,41,145,3,8,15,270,3,17,310,21,16,41,143,3,22,152,3,16,10,296,1,9,317,35,32,22


## FORMATTING

Converting Goals to our binary classification (0-1) and (2+) and also converting the sos and rpi and implementing the team to a particular rpi quartile

In [31]:
# Helper Function - Removes Columns to Ignore and Splits the Target Column
def split_target(data):
    td = model_libs._clone_and_drop(data, ignore_cols)
    (y, X) = model_libs._extract_target(td, target_col)
    return X, y

""" Need to do some formatting of the Data before we run the models"""
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 1000)

test = False
if test:
    ranked_data = form_data.get_rankings(leagues, teams, league_rounds, raw_data, False)
    ranked_data.to_csv('ranked_data.csv')          
else:
    ranked_data = pd.read_csv('ranked_data.csv')
    ranked_data = ranked_data.drop(ranked_data.columns[[0]], axis=1)

print('Ranked Data Loaded...')

Ranked Data Loaded...


### RUNNING CLASSIFICATION MODEL ON IF TEAMS SCORE 0-1 OR 1-2 ON GAMES

In [102]:
def run_features(data, drop_data, target, models):
    
    new_data = data.drop(drop_data, axis=1)
    #display(new_data.head())
    (y, X) = model_libs._extract_target(new_data, target)
    models = form_model.train_models(round_number, X, y, models)
    return models

""" Formatting data to convert goals scored to the correct category"""
formatted_data = ranked_data.copy()

if target == "converted_goals":
    # Not using points as a target for this version, using goals
    formatted_data['converted_goals'] = formatted_data.apply(lambda row: model_libs.set_group(row['goals']), axis=1)
    formatted_data = formatted_data.drop(['points', 'goals'], 1)
else: 
    formatted_data = formatted_data.drop(['goals'], 1)

""" //////////////////////////////////////////////////////////////////////////////////////////////////// """
""" Using diff_squared methods for features """
""" //////////////////////////////////////////////////////////////////////////////////////////////////// """
formatted_data["diff_goals_for"] = formatted_data.apply(lambda row: model_libs.diff_square(row["goals_for"], row["opp_goals_for"]), axis=1)
formatted_data["diff_goals_allowed"] = formatted_data.apply(lambda row: model_libs.diff_square(row["goals_against"], row["opp_goals_against"]), axis=1)
formatted_data["diff_attacks"] = formatted_data.apply(lambda row: model_libs.diff_square(row["current_team_attacks"], row["opp_team_attacks"]), axis=1)   
formatted_data["diff_dangerous_attacks"] = formatted_data.apply(lambda row: model_libs.diff_square(row["current_team_dangerous_attacks"], row["opp_team_dangerous_attacks"]), axis=1)   
formatted_data["diff_goal_attempts"] = formatted_data.apply(lambda row: model_libs.diff_square(row["current_team_goal_attempts"], row["opp_team_goal_attempts"]), axis=1)
formatted_data["diff_ball_safe"] = formatted_data.apply(lambda row: model_libs.diff_square(row["current_team_ball_safe"], row["opp_team_ball_safe"]), axis=1)                                                                                                                                                           
formatted_data["diff_possession"] = formatted_data.apply(lambda row: model_libs.diff_square(row["current_team_possession"], row["opp_team_possession"]), axis=1)
formatted_data["diff_corner_kicks"] = formatted_data.apply(lambda row: model_libs.diff_square(row["current_team_corner_kicks"], row["opp_team_corner_kicks"]), axis=1)                                                                                                                                                           
formatted_data["diff_goal_kicks"] = formatted_data.apply(lambda row: model_libs.diff_square(row["current_team_goal_kicks"], row["opp_team_goal_kicks"]), axis=1)
formatted_data["diff_saves"] = formatted_data.apply(lambda row: model_libs.diff_square(row["current_team_saves"], row["opp_team_saves"]), axis=1)

columns_to_drop = ['current_record', 'opp_record', 'goals_for', 'opp_goals_for', 'goals_against', 'opp_goals_against', 'rpi']

_, stats = form_data.get_columns()

formatted_data = formatted_data.drop(ignore_cols + columns_to_drop + stats, 1)
print(formatted_data.columns)
#### Running ALL Features 
if target == "converted_goals":
    models_test_1 = run_features(formatted_data, [], 'converted_goals', ["knn"])
    (formatted_y, formatted_X) = model_libs._extract_target(formatted_data, 'converted_goals')
else:
    models_test_1 = run_features(formatted_data, [], 'points', ["knn"])
    (formatted_y, formatted_X) = model_libs._extract_target(formatted_data, 'points')


def check_accuracy(model, data_X, y):
    actual_y = pd.DataFrame(y.values, columns=['actual'])
    predictions = pd.concat([pd.DataFrame(model.predict(data_X), columns=['predictions']), actual_y], axis=1)
    predictions['accuracy'] = predictions.apply(lambda r: model_libs.predictions_diff(r['predictions'], r['actual']), axis=1)
    accuracy = np.divide(predictions['accuracy'].sum(), float(len(predictions['accuracy'])))
    print(accuracy)

for m in models_test_1:
    check_accuracy(m, formatted_X, formatted_y)

Index([u'is_home', u'points', u'offensive_ranking', u'opp_defensive_ranking',
       u'rpi_ranking', u'opp_rpi_ranking', u'diff_goals_for',
       u'diff_goals_allowed', u'diff_attacks', u'diff_dangerous_attacks',
       u'diff_goal_attempts', u'diff_ball_safe', u'diff_possession',
       u'diff_corner_kicks', u'diff_goal_kicks', u'diff_saves'],
      dtype='object')
-----------------------------------
Training K Neighbors Classifier Model
KNN Score on Training Set :: 0.63708086785
KNN Score on Test Set:: 0.291338582677
Finished K-Means Modeling
0.567823343849


In [104]:
model_results = []
for m in all_models:
    r = form_model.build_tuned_model(formatted_X, formatted_y, m)
    model_results.append(r)
    print('Accuracy :: ')
    check_accuracy(r[0], formatted_X, formatted_y)

Training LOG REG Model
Score on Training Set :: 0.499013806706
Score on Test Set :: 0.488188976378
Finished LOG REG Modeling
Accuracy :: 
0.496845425868
Training and Tuning SVC Model
[ 0.34251969  0.33992095]
Accuracy: 0.34 (+/- 0.00)
Finished SVC Modeling
Accuracy :: 
0.862776025237
-----------------------------------
Training K-Means Model
KNN Score :: 0.663366336634 for Training
KNN Score :: 0.302325581395 for Testing
KNN Score :: 0.641025641026 for Training
KNN Score :: 0.275590551181 for Testing
KNN Score :: 0.667322834646 for Training
KNN Score :: 0.333333333333 for Testing
KNN Score :: 0.622047244094 for Training
KNN Score :: 0.333333333333 for Testing
KNN Score :: 0.625984251969 for Training
KNN Score :: 0.301587301587 for Testing
Finished K-Means Modeling
Accuracy :: 
0.561514195584
-----------------------------------
Training Gaussian NB Model
[ 0.44660194  0.3627451   0.45544554  0.41584158  0.36      ]
Accuracy: 0.41 (+/- 0.08)
Finished Gaussian NB Modeling
Accuracy :: 
0.4

Cross Validating the SVC model with the PCA data to help prevent overfitting

In [105]:
prediction_models = form_model.load_models(['knn', 'svc', 'randomForest', 'gnb', 'log', 'gmm'])

Success :: Loaded - knn
Success :: Loaded - svc
Success :: Loaded - randomForest
Success :: Loaded - gnb
Success :: Loaded - log
Success :: Loaded - gmm


In [17]:
print('Upcoming matches')
#upcoming_matches, match_details = predict_matches.get_upcoming_matches()
#upcoming_matches.to_csv('upcoming_matches.csv')
upcoming_matches = pd.read_csv('upcoming_matches.csv')
upcoming_matches = upcoming_matches.drop(upcoming_matches.columns[[0]], axis=1)
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 500)
upcoming_data = predict_matches.predictions(upcoming_matches)
print('Loaded Upcoming Data...')

test = False
if test:
    upcoming_ranked_data = form_data.get_rankings(leagues, teams, league_rounds, upcoming_data, True)
    upcoming_ranked_data.to_csv('upcoming_ranked_data.csv')               
else:
    upcoming_ranked_data = pd.read_csv('upcoming_ranked_data.csv')
    upcoming_ranked_data = upcoming_ranked_data.drop(upcoming_ranked_data.columns[[0]], axis=1)

#display(upcoming_ranked_data.head())

Upcoming matches
Loaded Upcoming Data...


In [106]:
""" Formatting data to convert goals scored to the correct category"""
upcoming_formatted_data = upcoming_ranked_data.copy()

""" //////////////////////////////////////////////////////////////////////////////////////////////////// """
""" Using diff_squared methods for features """
""" //////////////////////////////////////////////////////////////////////////////////////////////////// """
upcoming_formatted_data["diff_goals_for"] = upcoming_formatted_data.apply(lambda row: model_libs.diff_square(row["goals_for"], row["opp_goals_for"]), axis=1)
upcoming_formatted_data["diff_goals_allowed"] = upcoming_formatted_data.apply(lambda row: model_libs.diff_square(row["goals_against"], row["opp_goals_against"]), axis=1)
upcoming_formatted_data["diff_attacks"] = upcoming_formatted_data.apply(lambda row: model_libs.diff_square(row["current_team_attacks"], row["opp_team_attacks"]), axis=1)   
upcoming_formatted_data["diff_dangerous_attacks"] = upcoming_formatted_data.apply(lambda row: model_libs.diff_square(row["current_team_dangerous_attacks"], row["opp_team_dangerous_attacks"]), axis=1)   
upcoming_formatted_data["diff_goal_attempts"] = upcoming_formatted_data.apply(lambda row: model_libs.diff_square(row["current_team_goal_attempts"], row["opp_team_goal_attempts"]), axis=1)
upcoming_formatted_data["diff_ball_safe"] = upcoming_formatted_data.apply(lambda row: model_libs.diff_square(row["current_team_ball_safe"], row["opp_team_ball_safe"]), axis=1)                                                                                                                                                           
upcoming_formatted_data["diff_possession"] = upcoming_formatted_data.apply(lambda row: model_libs.diff_square(row["current_team_possession"], row["opp_team_possession"]), axis=1)
upcoming_formatted_data["diff_corner_kicks"] = upcoming_formatted_data.apply(lambda row: model_libs.diff_square(row["current_team_corner_kicks"], row["opp_team_corner_kicks"]), axis=1)                                                                                                                                                           
upcoming_formatted_data["diff_goal_kicks"] = upcoming_formatted_data.apply(lambda row: model_libs.diff_square(row["current_team_goal_kicks"], row["opp_team_goal_kicks"]), axis=1)
upcoming_formatted_data["diff_saves"] = upcoming_formatted_data.apply(lambda row: model_libs.diff_square(row["current_team_saves"], row["opp_team_saves"]), axis=1)

columns_to_drop = ['current_record', 'opp_record', 'goals_for', 'opp_goals_for', 'goals_against', 'opp_goals_against', 'rpi']
_, stats = form_data.get_columns()

upcoming_formatted_data = upcoming_formatted_data.drop(ignore_cols + columns_to_drop + stats + ['points', 'goals'], 1)
print(upcoming_formatted_data.columns)

Index([u'is_home', u'offensive_ranking', u'opp_defensive_ranking',
       u'rpi_ranking', u'opp_rpi_ranking', u'diff_goals_for',
       u'diff_goals_allowed', u'diff_attacks', u'diff_dangerous_attacks',
       u'diff_goal_attempts', u'diff_ball_safe', u'diff_possession',
       u'diff_corner_kicks', u'diff_goal_kicks', u'diff_saves'],
      dtype='object')


In [107]:
""" Models we'll use to predict on upcoming matches """

# This is all the X values
upcoming_formatted_data

rf_preds = prediction_models[2].predict(upcoming_formatted_data)
print(rf_preds)

knn_preds = prediction_models[0].predict(upcoming_formatted_data)
print(knn_preds)

svc_preds = prediction_models[1].predict(upcoming_formatted_data)
print(svc_preds)

log_preds = prediction_models[3].predict(upcoming_formatted_data)
print(log_preds)

log_prob = prediction_models[3].predict_proba(upcoming_formatted_data)
probs = pd.DataFrame(log_prob)
#probs.to_csv('probs.csv')

gnb_preds = prediction_models[3].predict(upcoming_formatted_data)
print(gnb_preds)


[ 0.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  3.  3.  3.  1.  0.  0.  0.
  1.  1.  1.  1.  3.  0.  0.  1.  3.  0.  3.  0.  0.  0.  0.  0.  3.  3.
  1.  1.  1.  0.  3.  1.  1.  0.  3.  3.  3.  0.  3.  0.  1.  0.  0.  0.
  0.  3.  0.  1.  3.  0.  0.  0.  1.  3.  3.  3.  1.  3.  0.  1.  1.  1.
  3.  1.  0.  1.  0.  1.  0.  0.  0.  3.  0.  1.  3.  3.  1.  1.  1.  0.
  1.  0.  3.  3.  1.  1.  0.  3.]
[ 1.  0.  0.  1.  0.  0.  0.  1.  1.  1.  0.  3.  3.  3.  0.  0.  1.  0.
  0.  1.  1.  3.  0.  1.  0.  0.  0.  3.  1.  0.  0.  0.  0.  1.  1.  3.
  3.  0.  1.  1.  0.  0.  0.  0.  1.  1.  0.  0.  0.  0.  3.  0.  0.  0.
  1.  0.  0.  0.  1.  0.  0.  0.  0.  1.  0.  0.  0.  0.  3.  0.  1.  0.
  1.  1.  0.  0.  1.  0.  3.  1.  1.  1.  0.  0.  1.  1.  0.  3.  1.  3.
  0.  1.  0.  0.  0.  1.  0.  0.]
[ 3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.
  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.
  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.

In [108]:
columns = ['team_name', 'opp_name', 'scheduled', 'is_home']
# Remove all columns except the ones above
upcoming_matches = upcoming_data[columns]

random_preds = pd.Series(np.random.randint(2, size=len(upcoming_matches.index)), upcoming_matches.index)
# Add predictions to the end of that DF
results = pd.DataFrame({'KNN': knn_preds, 'RandomForest': rf_preds, 'SVC': svc_preds, 'GNB': gnb_preds, 'log': log_preds, 'random': random_preds})
upcoming_matches = pd.concat([upcoming_matches, results], axis = 1)
reordered_matches = pd.DataFrame([])

for rows in upcoming_matches.iterrows():
    for i in upcoming_matches['team_name']:
        if rows[1]['opp_name'] == i:
            reordered_matches = reordered_matches.append(rows[1])
            reordered_matches = reordered_matches.append(upcoming_matches[upcoming_matches['team_name'].isin([i])])

reordered_matches = reordered_matches.drop_duplicates() 
columns = ['scheduled', 'team_name', 'opp_name', 'is_home', 'KNN', 'RandomForest', 'SVC', 'GNB', 'log', 'random']
reordered_matches = reordered_matches[columns]
actual_results = pd.read_csv('actual_results.csv')
actual_results=actual_results.rename(columns = {'Unnamed: 0':'idx'})
indexed_results = actual_results.set_index('idx')
reordered_matches = pd.concat([reordered_matches, indexed_results], axis=1)
reordered_matches = reordered_matches.reset_index(drop=True)
display(reordered_matches.head(5))
reordered_matches.to_csv('predictions_converted_goals.csv')
print('Prediction CSV saved')

,scheduled,team_name,opp_name,is_home,KNN,RandomForest,SVC,GNB,log,random,actual,actual_converted_goals
0,2016-09-25 01:30:00,FC Dallas,Real Salt Lake,0,1,0,3,0,0,0,1,0
1,2016-09-25 01:30:00,Real Salt Lake,FC Dallas,1,0,1,3,3,3,1,1,0
2,2016-09-24 23:00:00,Colorado Rapids,Vancouver Whitecaps FC,0,0,1,3,1,1,0,1,1
3,2016-09-24 23:00:00,Vancouver Whitecaps FC,Colorado Rapids,1,1,0,3,1,1,1,1,1
4,2016-09-24 00:00:00,New York City FC,Chicago Fire,1,1,0,3,3,3,1,3,1


Prediction CSV saved


In [114]:
if target == 'converted_goals':
    actual = "actual_converted_goals"
    reordered_matches = pd.read_csv('predictions_converted_goals.csv')
    reordered_matches = reordered_matches.drop(reordered_matches.columns[[0]], axis=1)
    reordered_matches['accuracy'] = reordered_matches.apply(lambda r: model_libs.predictions_diff(r[actual], r['SVC']), axis=1)
    accuracy = np.divide(reordered_matches['accuracy'].sum(), float(len(reordered_matches['accuracy'])))
    print(accuracy)
else:
    actual = "actual"
    results_data = reordered_matches.copy()
    home_actual_win = 0
    home_predicted_win = 0
    predicted_draws = 0
    actual_draws = 0
    valid_matches = 0
    total_matches = 0
    invalid_predictions = []
    correct_predictions = 0
    column_model = "GNB"

    for r, rows in results_data.iterrows():
        if r % 2 == 0:
            total_matches += 1
            predictions = results_data.loc[r:r+1, ["is_home", column_model, actual]]
            home_team = predictions[predictions["is_home"] == 1]
            #print(predictions)
            if home_team.iloc[0]["actual"] == 3:
                home_actual_win += 1
            elif home_team.iloc[0]["actual"] == 1:
                actual_draws += 1

            # Check if it's a valid prediction (0-3, 3-0, 1-1)
            is_valid = False
            if ((predictions.iloc[0][column_model] == 1) & (predictions.iloc[1][column_model] == 1)) or ((predictions.iloc[0][column_model] == 3) & (predictions.iloc[1][column_model] == 0)) or ((predictions.iloc[0][column_model] == 0) & (predictions.iloc[1][column_model] == 3)):
                is_valid = True
                valid_matches += 1

                if home_team.iloc[0][column_model] == 3:
                    home_predicted_win += 1

                    if home_team.iloc[0]["actual"] == 3:
                        correct_predictions += 1

                if (predictions.iloc[0][column_model] == 1) & (predictions.iloc[1][column_model] == 1):
                    predicted_draws += 1

                    if home_team.iloc[0]["actual"] == 1:
                        correct_predictions += 1
            else:
               invalid_predictions.append(predictions) 

    print(column_model)        
    print('Total Matches :: {}'.format(total_matches))
    print('Valid Predicted Matches :: {}'.format(valid_matches))
    print('Actual Home Team Wins :: {}'.format(home_actual_win))
    print('Home Predicted Wins :: {}'.format(home_predicted_win))
    print('Actual Draws :: {}'.format(actual_draws))
    print('Predicted Draws :: {}'.format(predicted_draws))
    print('Correct Predictions :: {}'.format(correct_predictions))
    
    results_data['accuracy'] = results_data.apply(lambda r: model_libs.predictions_diff(r[column_model], r[actual]), axis=1)
    accuracy = np.divide(results_data['accuracy'].sum(), float(len(results_data['accuracy'])))
    print('Individual Accuracy :: {}'.format(accuracy))

    #print(invalid_predictions)
    

GNB
Total Matches :: 49
Valid Predicted Matches :: 43
Actual Home Team Wins :: 26
Home Predicted Wins :: 17
Actual Draws :: 9
Predicted Draws :: 6
Correct Predictions :: 10
Individual Accuracy :: 0.428571428571
